# Python Asyncrhonous Programming

[https://blog.humminglab.io/python-coroutine-programming-1/](https://blog.humminglab.io/python-coroutine-programming-1/)

[https://blog.humminglab.io/python-coroutine-programming-2/](https://blog.humminglab.io/python-coroutine-programming-2/)

* Python 3.4: `asynchio` module added
* Ptyhon 3.5: `async`, `await` keywords added for `native coroutine`

Javascript나 다른 언어에서 지원하는 비동기 프로그래밍의 장점을 Python에서도 사용할 수 있다. 즉, 이벤트 방식이지만 blocking 방식의 프로그래밍처럼 sequential 하게 코드를 작성할 수 있어, 단일 thread로 수만개의 네트워크 연결을 처리하는 서버를 오류 가능성을 최소화하면서 보다 편하게 개발.

다음과 같은 순서로 설명
1. `iterator`
2. `yield` & `generator`
3. `yield from`
4. `asyncio`
5. `async` & `await`


## Iterator

`List` 와 같은 데이터구조는 수십만개의 저장공간이 필요할 경우 잠재적인 메모리 부족문제를 만들 수 있다.
하지만 Python3의 `range()`함수의 경우 `iterator`를 리턴하여 실제로 데이터가 할당되지 않는다.

`Iterator`: 이전 원소로 다음 원소를 계산할 수 있는 Data structure

* `__iter__()`: `Iterator` object을 리턴
* `__next__()`: 호출될때마다 다음값 리턴

Python 2.1부터 [PEP234 - Iterators](https://www.python.org/dev/peps/pep-0234/) 지원.

In [1]:
class Counter(object): # where run `next(Counter())` : TypeError: 'Counter' object is not an iterator
    def __iter__(self):
        iter = Iterator()
        return iter

class Iterator(object):
    def __init__(self):
        self.index = 0

    def __next__(self):
        if self.index > 10:
            raise StopIteration
        n = self.index * 2
        self.index += 1
        return n

In [2]:
cc = Iterator()
print(next(cc))
print(next(cc))

0
2


In [3]:
c = Counter()
for i in c:
    print(i)


0
2
4
6
8
10
12
14
16
18
20


## Generator

`yield`를 이용하여 `coroutine`을 지원한다. Python 2.2, [PEP 255 - Simple Generator](https://www.python.org/dev/peps/pep-0255/)에서는 lightweight coroutine 지원이 추가되었으며, 이를 `generator`라고 부름.

**Coroutine이란?**
[https://wooooooak.github.io/kotlin/2019/08/25/코틀린-코루틴-개념-익히기/](https://wooooooak.github.io/kotlin/2019/08/25/코틀린-코루틴-개념-익히기/)
1. 협력형 멀티 태스킹
2. 동시성 프로그래밍 지원
3. 비동기 처리를 쉽게 도와줌


In [4]:
def test1():
    print('print 1')
    yield 1
    print('print 2')
    yield 2

def test2():
    for i in range(5):
        yield i * 2

Generator는 함수 안에 `yield` 가 있으며, 호출시, Generator가 실행되는 것이 아니라 이 함수를 감싸는 `generator` 객체를 리턴.

Iterator와 동일하게 `__next__()`를 가진다.

In [5]:
g = test1()
print(type(g))
print(dir(g))
print(next(g))
print(next(g))
print(next(g))


<class 'generator'>
['__class__', '__del__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__name__', '__ne__', '__new__', '__next__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'close', 'gi_code', 'gi_frame', 'gi_running', 'gi_yieldfrom', 'send', 'throw']
print 1
1
print 2
2


StopIteration: 

In [ ]:
gg = test2()
print(next(gg))
print(next(gg))
print(next(gg))
print(next(gg))
print(next(gg))
print(next(gg))


이과 같이 Generator는 `yield`를 이용하여 중간에서 멈추고 결과를 받는 용도로 사용.
실제 주요 용도는 위의 `test2()`와 같이 Iterator를 쉽게 만드는 것.

Iterator와 Generator는 생성과정은 다르지만, 생성된 후 동작은 완전히 동일.


Python2.4원, [PEP 289 - Generator Expressions](https://www.python.org/dev/peps/pep-0289/) 에서 더 간결한 generator 생성 지

In [ ]:
# return list
print([x * 2 for x in range(10)])

In [ ]:
# return generator
(x* 2 for x in range(10))

**Exception 처리**

Generator 안에서 동작 중 발생하는 exception은 next()를 호출한 부분으로 정상적으로 전달 후, 종료.

* 한번 exception이 발생한 generator는 더이상 실행할 수 없다. (StopIteration)
* generator 함수에서는 try/except는 사용가능하나, try/except/finally 사용불능. (Yield 기반의 coroutine으로 확장 되면서 try/finally는 가능.)

In [ ]:
def f():
    return 1/0
def g():
    yield f()   # the zero division exception propagate
    yield 42    # and we'll never get here

In [ ]:
k = g()
next(k) # ZeroDivisionError

In [ ]:
next(k) # StopIteration


In [ ]:
def f():
    try:
        yield 1
        try:
            yield 2
            1/0     # ZeroDivisionError
            yield 3 # never get here
        except ZeroDivisionError:
            yield 4
            yield 5
            raise
        except: # the "raise" above stops this
            yield 6
            yield 7
    except:
        yield 8
    yield 9
    try:
        x = 12
    finally:
        yield 10
    yield 11

In [ ]:
print(list(f()))


## Yield 기반의 Coroutine

generator의 `yield`는 generator 함수 내의 코드에서 호출된느 곳으로 데이타를 전달하는 것으로 볼 수 있다.
즉, `yield`가 실행될 때 마다 해당 함수는 멈추고, 값을 `next(x)`를 호출한 함수로 전달.

참고: 이는 `yield` 기반의 coroutine은 generator와 동일하다고 보면 된다.
Python 3.5에서는 `async`로 `coroutine`를 지원하는데, 이는 native coroutine이라고 한다.

In [ ]:
def callee():
    yield 1
    yield 2

x = callee()    # caller
i = next(x)
i = next(x)

* `next()`가 호출되는 시점에서 제어권을 generator로 넘김
* generator 내의 코드는 `yield`를 만날때까지 실행되고, 이때의 값을 호출한 곳으로 전달.

하지만, 데이타는 callee에서 caller로만 전달된다.

이를 보완하여, 상호간에 데이타와 함께 제어권을 전달하는 방법이 Python 2.5의 `yield`기반으로 확장된 coroutine이다.
[PEP 342 - Coroutine via Enhanced Generators](https://www.python.org/dev/peps/pep-0342/#new-generator-method-send-value)

이를 통해, Python은 단일 thread로 다수의 작업(coroutine)을 concurrent하게 실행할 할 수 있는 coroutine을 갖게됨.
* ```x = yield 1```와 같이 `yield` 키워드에서도 값을 받을 수 있다.
* `next()`가 아닌 `send()`함수가 추가되어 이를 이용하여 caller는 coroutine의 `yield`에 값을 전달
* coroutine 실행하는 동안 exception 처리를 지원

In [ ]:
def coroutine1():
    print('callee 1')
    x = yield 1
    print('callee 2: %d' %x)
    x = yield 2
    print('callee 3: %d' %x)

task = coroutine1()
i = next(task)          # callee 1, i = 1
i = task.send(10)       # callee 2: 10, i = 2
task.send(20)           # callee 3: 20, StopIteration exception

generator의 경우, generator에서 caller로는 try-finally제한은 있지만 exception을 전달 할 수 있다.
또한 coroutine의 마지막 라인까지 실행되면 StopIteration Exception으로 종료를 알릴 수 있다.
그러나, 반대로 caller에서 coroutine으로 exceptin을 전달할 방법이 없고, caller를 parent task라는 개념으로 봤을 때 child인 coroutine을 종료시킬 수 없다.
이를 지원하기 위해 다음과 같은 사항 추가됨
* generator에서 yield문은 try-finally로 감쌀 수 없었으나, Python 2.5부터는 이를 지원한다. 이렇게 되면 coroutine > caller로의 exception 전달 지원
* caller에서 yield(또는 생정직후)로 멈춰있는 coroutine에 exception 전달 지원. `send()`와 비슷한 방법으로 `throw(type, value, traceback)`처럼 exception을 coroutine에 전달. parameter는 raise의 parameter와 동일.
* caller에서 coroutine을 종료시킬 수 있는 `close()`도 추가됨. 이를 위하여 GeneratorExit exception이 추가됨. `close()`는 `throw()`를 이용하여 GeneratorExit exception을 coroutine에 전달.

이를 통해 비동기방식의 프로그래밍을 위한 부분이 완료.

## yield from

Python3부터 지원되는 기능들

일반 함수나 task도 1:1 통신을 하는 경우 말고, a <-> b <-> c와 같이 통신을 하는 경우도 많다. (task가 sub-task를 재호출하는 방식)
coroutine도 마찬가지. coroutine이 다시 sub-coroutine을 호출하는 구조가 될 수 있다.

In [ ]:
def subcoroutine():
    yield 1
    yield 2

def coroutine():
    for v in subcoroutine():
        yield v

x = coroutine()
print(next(x))  # 1
print(next(x))  # 2
next(x)         # StopIteration

하지만 여기에 추가된 `send()`, `throw()`, `close()`를 지원하려면 다음과 같이 수정되어야 한다.

In [ ]:
def subcoroutine():
    print("Subroutine")
    x = yield 1
    print("Recv: " + str(x))
    x = yield 2
    print("Recv: " + str(x))

def coroutine():
    _i = subcoroutine()
    _x = next(_i)
    while True:
        _s = yield _x

        if _s is None:
            _x = next(_i)
        else:
            _x = _i.send(_s)

x = coroutine()
print(next(x))
print(x.send(10))
print(x.send(20))

중간에 `_s`의 값을 보고 `next()`와 `send()`를 구분하는데, 위 코드에서는 그냥 `_x = _i.send(_s)`를 호출해도 된다.
`send(None)`은 `next()`와 동일하다. 하지만  `_i`가 위처럼 generator가 아니라 iterator라면 반드시 `next()`를 해주어야 한다.
(iterator는 `next()`만 구현되고, `send()`는 없다.)
여기에 `throw()`, `close()`등의 exception 사항을 추가하면 복잡한 일.

이것을 한줄로 지원하는것이 Python 3.3, [PEP380 - Syntax for Delegating to a Subgenerator](https://www.python.org/dev/peps/pep-0380/)에 추가된 `yield from`.

위와 같이 corountine에서 sub-coroutine을 호출하여 결과적으로 caller <-> sub-coroutine이 데이타를 주고 받게 하려면 위와같이 복잡하게 하지 않고 `yield from`을 사용하면 된다.

`yield from`을 이용하면 위의 예제는 다음과 같음

In [ ]:
def subcoroutine():
    print("Subcoroutine")
    x = yield 1
    print("Recv: " + str(x))
    x = yield 2
    print("Recv: " + str(x))

def coroutine():
    yield from subcoroutine()

`yield from` 오른쪽에 들어갈 수 있는 것은 iterable과 generator이다.

[PEP 380](https://www.python.org/dev/peps/pep-0380/)에 하나 더 추가된 기능은, generator에서 return 지원이다.
기존의 generator에서는 return을 사용하여 값을 반환할 수 없고, 종료되면 StopIteration exception이 발생하는 방식이었다.
이를 return으로 값과 StopIteration을 동시에 반환하도록 기능을 추가하였다.

In [ ]:
def test():
    yield 1
    return 10

# def test():
#     yield 1
#     e = StopIteration()
#     e.value = 10
#     raise e

In [ ]:
x = test()
print(next(x))
print(next(x))  # return 10 & StopIteration



위와 같은 `return`을 추가로 지원하는 것은 `yield from`의 기능을 확장하기 위한것이다.
이런 구조가 되면 lightweight thread 형태가 된다.
즉, sub-coroutine은 필요한 만큼 돌다가 결과가 나오면 그때 반환하는 방식이다.

In [ ]:
def sum(max):
    tot = 0
    for i in range(max):
        tot += i
        yield tot
    return tot

def coroutine():
    x = yield from sum(10)
    print('Total: {}'.format(x))

x = coroutine()
print(next(x))
print(next(x))
print(next(x))
print(next(x))
print(next(x))
print(next(x))
print(next(x))
print(next(x))
print(next(x))
print(next(x))
print(next(x))




위의 `sum()`과 같은 sub-coroutine에서 `yield`로 주는 값과 `return`되는 값은 용도가 다르다.
`yield tot`로 전달한 것은 coroutine를 거쳐 caller에서 받는다. 하지만 `yield from`으로 받은 값은 caller가 아니라 중간에 있 parent인 coroutine에서 받는다.
즉, 두가지는 사용하는 용도가 다르다.
`yield tot`과 같이 `coroutine`에서의 값을 caller로 전달하는 것은 iterator의 확장이라고 볼 수 있다.
하지만, 보통 coroutine 기반으로 lightweight thread를 작성하는 경우 이와 같이 중간값 보다는 다른 coroutine과 같이 돌다가 최종 결과를 받기를 원하는 경우이다.
이때 `yield from`을 통한 `return`값을 사용한다.


## Asyncio
Python3.4에서는 그동안 Twisted, Tornado와 같이 별도의 library로 제공되던 event loop방식의 비동기 프로그래밍이
ayncio, [PEP 3156 - Asynchronous IO Supported Rebooted: the "asyncio" module](https://www.python.org/dev/peps/pep-3156/) 표준 라이브러리로 추가됨.

이 event loop는 C언어에서 사용하는 것과 같은 callback 방식으로도 사용은 가능하지만, coroutine과 같이 사용한다면 큰 장점 발휘.
단일 thread에 마치 multi-tasking을 하는 것과 같은 유사한 기능을 수행할 수 있다.

**future**

이를 이해하기 위하여 먼저 기존 Python의 thread나 process에서의 concurrent 프로그래밍 방식으로 Python3.2에 추가된 `future`, [PEP 3148 -- futures - execute computations asynchronously](https://www.python.org/dev/peps/pep-3148/) 를 이해할 필요가 있다.
이후에 만들어진 asyncio도 이와 동일한 API로 만들어진 것이다.

`future`는 쉽게 말해서 work thread(process)의 핸들이라고 볼 수 있다. 이를 `future.result()`와 같이 종료가 끝날 때까지 기다리게 하면, 해당 work function에서 결과를 완료하거나, exception이 발생한 경우 이를 받을 수 있다.

In [ ]:
from concurrent import futures
import urllib.request

URLS = ['http://foxnews.com/',
        'http://www.cnn.com',
        'http://some-made-up-domain.com/']

def load_url(url, timeout):
    return urllib.request.urlopen(url, timeout=timeout).read()

def main():
    with futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_to_url = dict(
            (executor.submit(load_url, url, 60), url) for url in URLS)

        for future in futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                print('%r page is %d bytes' %(url, len(future.result())))
            except Exception as e:
                print('%r generated an exception: %s' % (url, e))

if __name__ == '__main__':
    main()

* `executor.submit()`으로 thread pool에서 돌릴 함수를 등록하면 `future`를 리턴한다.
등록된 함수는 thread pool에서 비동기로 실행된다.

* `futures.as_completed()`와 같이 결과를 완료된 순서되로 리턴되는 generator를 리턴받을 수 있다.
위와 같이 `for ... in`에 넣어 loop을 돌릴 수 있다. 완료되거나 비정상 종료된 future가 차례대로 나오게 된다.

* `future.result()`로 결과를 받을 수 있다. 만일 `future` 내의 함수, `load_url()`, 에서 exception이 발생한 것도 future를 통하여 호출한 thread에서 받을 수 있다.
위의 예제와 같이 `future.result()`를 `try...except`문으로 감싸서 해당 작업에서 발생한 exception도 받을 수 있다.

Asyncio에서도 `concurrent.futures.Future`와 유사한 `asyncio.future`를 제공한다. 차이점은 일반 함수가 아니라 coroutine을 전달하는 것이고,
`future.result()`함수가 blocking이 되지 않는다 정도의 차이가 있다.

`Asyncio`를 이해하는데 혼동이 되는 부분이 이 `future`때문인데, 사용방법을 정리하면 다음과 같다.

* Asyncio는 future 없이도 callback만 사용가능하다. 이 경우 `call_later()`등의 함수을 이용하여 callback을 등록하고 사용할 수 있다.
하지만, 이것만 사용하게 되면 진정한 asyncio의 장점을 살릴 수 없게 된다. `call_later()`등을 이용하여 callback을 등록하면 event loop에서 적절한 시점에 callback을 호출한다.

* Coroutine은 future를 이용. 실제적으로 future을 직접 사용하지 않고, 이를 상속받은 Task class를 사용한다.
Future와 Task의 차이점은 Future은 coroutine의 예외 처리를 위해 감싼것이고, Task는 여기에 event loop와 연한 것.

참고: callback이란? argument로 다른 함수에게 전달되는 함수, 나중에 실행됨.
[https://beconstant.tistory.com/2](https://beconstant.tistory.com/2)



여기서 나오는 `future`, `Task`, `coroutine`, `yield from`, ` @asyncio.coroutine` 등의 용어들을 명확히 이해해보자

` @asyncio.coroutine`는 decorator이다. 하지만 이는 실제로 특별한 기능을 수행하지 않고, asyncio와 같이 사용하는 coroutine이라고 표하는 documentation 목적이다.


함수 앞에 @를 쓰면 decorator라 한다. 이는 함수를 parameter로 받아서 다시 함수를 리턴하는 함수이다. 기존 함수를 변형하는 용도로 사용한다.
예를 들어 기존 함수를 decorator를 이용하여 함수의 입출력을 바꾸거나, trace 등을 할 수 있다.

일반적으로 caller에서 반복적으로 `next()`, `send()`를 이용하여 `yield`에 멈춰있는 coroutine을 재개시킨다.
couroutine A에서는 내부적으로 다시 coroutine B를 호출 할 수 있다.
이때는 편리하게 `yield from`으로 호출하면 호출된 coroutine B가 `yield`가 반복되어 최종 `return`될때까지 coroutine A는 기다리게 된다.
결국, caller, coroutine A, coroutine B를 같이 놓고 보면 caller가 `send()`를 호출할 때마다 coroutine B의 yield가 풀리는 셈이다.

`send()`를 반복적으로 호출하는 것은 asyncio의 event loop에서 한다고 보면 된다.
이렇게 되면 coroutine도 event loop에서 마치 별도의 thread에서 도는 것과 같이 실행되는 셈이 된다.
이 coroutine을 event loop에서 관리하기 위하여 `future`를 상속받은 `task`를 사용하는 것이다.

일반 callback 함수는 `call_later()`로 event loop에 등록하고, coroutine은 `ensure_future()`나 `loop.create_task()`를 사용하여 등록.

Asyncio를 사용한 예제는 [asyncio.readthedocs.io](http://asyncio.readthedocs.io/)를 참조하면 다양한 사용 예제가 있다.


```python
import asyncio

@asyncio.coroutine
def print_every_second_coroutine(type):
    "Print seconds"
    while True:
        for i in range(10):
            print(i, 's (corotine {})'.format(type))
            yield from asyncio.sleep(1)
        loop = asyncio.get_event_loop()
        loop.stop()

def print_every_seconds_callback(i):
    print (i, 's (callback)')
    loop = asyncio.get_event_loop()
    loop.call_later(1.0, print_every_seconds_callback, i+1)

def print_every_seconds_callback_to_coroutine():
    asyncio.ensure_future(print_every_second_coroutine('B'))

loop = asyncio.get_event_loop()
loop.call_soon(print_every_seconds_callback, 0)
loop.call_soon(print_every_seconds_callback_to_coroutine)
asyncio.ensure_future(print_every_second_coroutine('A'))

loop.run_forever() # it's not working...
loop.close()
```

* `print_every_second_coroutine()`은 `asyncio.ensure_future()`를 이용하여 default event handler에 coroutine을 등록.
이때는 generator나 future를 등룍하여야하기 때문에 `print_every_second_coroutine('A')`와 같이 generator를 리턴받아서 등록한다.
이는 event loop, `loop.run_forever()`에서 호출 된다.

* callback은 `print_every_seconds_callback`과 같이 함수 이름을 전달한다. 기본적으로 one shot이기 때문에 callback 함수에서는 `call_later()`등의 method를 이용하여 반복해서 호출.

* `print_every_seconds_callback_to_coroutine()`과 같이 일반 callback 함수에서는 coroutine을 직접 호출할 수 없다.
(직접 호출하려면 이 함수가 `next()`를 반복해서 호출하여야 하기 때문에 event loop blocking 됨.
대신, coroutine을 등록하는 것과 동일하게 `asyncio.ensure_future()` 또는 `loop.create_task()`를 사용한다.


## async, await

Python3.5에서는 coroutine을 명시적으로 지정하는 `async`와 `yield`를 대체하는 `await` keyword가 추가되었다.
[PEP 492 - Coroutines with async and await syntax](https://www.python.org/dev/peps/pep-0492/)
이를 기존의 `yield`를 기반으로 하는 generator based coroutine과 비교하기 위하여 native coroutine이라고 한다.

native coroutine은 함수 앞에 `async def`를 붙여 사용.

In [ ]:
async def read(db):
    pass

`async` 함수는 기존문법인 `yield`, `yield from`을 사용할 수 없고, `await`를 사용한다.
또한 위와 같이 함수 안에 `await`를 사용하지 않아도 `async def`로 정의된 함수는 coroutine이 된다.

다음과 같은 사항이 `await` 오른쪽에 올 수 있다.
* native coroutine object
* 기존의 generator based coroutine object (정확히는 새로 추가된 ` @types.coroutine` decorator를 붙인 generator)
* `__await__` method를 가진 object를 리턴하는 iterator.
* CPython API를 위한 `tp_as_async.am_await`

일단 `yield`로 두개가 concurrent하게 돌아가는 event loop을 최소로 만들어 보면 다음과 같다.

In [4]:
def coro1():
    print('C1: Start')
    yield
    print('C1: a')
    yield
    print('C1: b')
    yield
    print('C1: end')

def coro2():
    print('C2: Start')
    yield
    print('C2: a')
    yield
    print('C2: b')
    yield
    print('C2: end')

def run(coros):
    coros = list(coros)

    while coros:
        for coro in list(coros):
            try:
                coro.send(None)
            except StopIteration:
                coros.remove(coro)

c1 = coro1()
c2 = coro2()
run([c1, c2])

C1: Start
C2: Start
C1: a
C2: a
C1: b
C1
C1
C2: b
C1: end
C2: end


이를 `async`/`await`로 바꿔보면 우선 첫번째 문제는 위 `yield`처럼 뒤에 operand없이 그냥 `await`만을 사용이 안된다.
`await`만으로 동일한 switch loginc을 만들 수 없어, 기존 generator based coroutine으로 task switching을 하도록 구현한다.

In [5]:
import types

@types.coroutine
def switch():
    yield

async def coro1():
    print('C1: Start')
    await switch()
    print('C1: a')
    await switch()
    print('C1: b')
    await switch()
    print('C1: end')

async def coro2():
    print('C2: Start')
    await switch()
    print('C2: a')
    await switch()
    print('C2:b')
    await switch()
    print('C2: end')

def run(coros):
    coros = list(coros)

    while coros:
        for coro in list(coros):
            try:
                coro.send(None)
            except StopIteration:
                coros.remove(coro)

c1 = coro1()
c2 = coro2()
run([c1, c2])


C1: Start
C2: Start
C1: a
C2: a
C1: b
C2:b
C1: end
C2: end


Python 3.5에서 예외 처리도 보완 되었는데, 중첩된 coroutine에서 StopIteration이 발생 시 어느 것의 exception인지 처리가 모호해지는 문제를 위하여
coroutine 밖으로 전파될때는 StopIteration이 RuntimeError로 변경 (PEP 479 - Change StopIteration handling inside generators)

**Async Interal**

`async`, `await`를 좀 더 깊게 들어가보자.

우선 이와 같이 `async`/`await`가 추가되면서 확장된 data model을 보면 다음과 같다.

* Awaitable object

    * `__await__()`가 구현된 객체. `async def`함수를 호출하여 return되는 native coroutine이 awaitable object.
    * `object.__await__(self)`에서 Iterator가 return되어, `await`에서 사용된다. Future의 경우도 `__await__()`가 구현되어서 `await`에 사용할 수 있는 것.

* Coroutine object

    * Awaitable object. 여기에 `coroutine.send(value)`, `coroutine.throw(type[, value[, tracebak]])`, `coroutine.close()`이 구현.

* Asynchronous Iterators

    * 기존의 iterator와 비슷하게 `__aiter__()`, `__anext__()` method가 구현된 객체.
    * 이 객체는 새로 추가된 `async for`에 사용할 수 있다.

* Asynchronous Context Managers

    * 기존에 `with`에서 사용하던 객체와 비슷하게 `__aenter__()`, `__aexit__()` method가 구현된 객체
    * 새로 추가된 `async with`에 사용할 수 있다.

**Asynchronous Context Manager**

Asynchronous Context Manager는 기존의 `with`를 `async` 버전으로 만든 것이라고 보면 된다.
```python
async with lock:
    ...
```

이를 기존의 `yield from`를 사용하여 표현하면
```python
with (yield from lock):
    ...
```

이 기능이 어떻게 풀어지는지 보면
```python
async with EXPR as VAR:
    BLOCK
```

위의 문장은 다음과 같은 code와 동일하게 실행된다.
```python
mgr = (EXPR)
aexit = type(mgr).__aexit__
aenter = type(mgr).__aenter__(mgr)

VAR = await aenter
try:
    BLOCK
except:
    if not await aexit(mgr, *sys.exc_info()):
        raise
else:
    await aexit(mgr, None, None, None)
```

`with`문장이 시작될때 `__aenter__()`가 호출되어 전처리 작업이 실행되고, exception 등의 모든 조건에서도 자동으로 `__aexit__()`가 호출되어 후처리가 되는 것.

**Asynchornous Generator**

Python 3.6, [PEP 525 - Asynchronous Generators](https://www.python.org/dev/peps/pep-0525/)를 보기를 추천.
Asynchronous generator는 간단히 말해서 `async`문에도 `yield`를 사용하여 data producer를 할 수 있는 generator 기능을 추가하는 것.

우선 기존의 `yield`문법으로 network에서 데이타를 받는 generator를 만들면 다음과 같은 구조일 것이다. 네트워크 함수는 blocking mode인 셈.
```python
def get_docs():
    page = fetch_page()
    while page:
        for doc in page:
            yield doc
        page = fetch_page()

for doc in get_docs():
    pass # work on doc
```

`for`문이 generator를 받아서 루프를 돌면 doc은 get_docs()에서 보내오는 doc 데이타를 전달 받게 된다.
이를 비동기 함수로 교체하고 적절히 `async`를 분이면

```python
async def get_doct():
    page = await fetch_page()
    while page:
        for doc in page:
            yield doc
        page = await fetch_page()

async for doc in get docs():
    pass # work on doc

```

참고로 `async for`는 다음과 같은 문장과 같은 의미를 가진다.

```python
async for TARGET in ITER:
    BLOCK
else:
    BLOCK2
```

```python
iter = (ITER)
iter = type(iter).__aiter__(iter)
running = True
while running:
    try:
        TARGET = await type(iter).__anext(iter)
    except StopAsyncIteratioin:
        running = False
    else:
        BLOCK
else:
    BLOCK2

```
